In [171]:
import pandas as pd
import numpy as np
from itertools import combinations

In [172]:
data_by_product = pd.read_csv('data_by_product.csv')
data_by_transaction = pd.read_csv('data_by_transaction.csv')

display(data_by_transaction)
display(data_by_product)

rules = pd.Series(index=np.arange(1, 4))

,1,2,3,4,5,6,7,8,9,10,11,12
0,yogurt,toilet paper,soda,cereals,sandwich loaves,laundry detergent,individual meals,ice cream,juice,ketchup,sandwich loaves,pork
1,pork,shampoo,pork,juice,pasta,toilet paper,paper towels,juice,poultry,coffee/tea,ice cream,tortillas
2,sandwich bags,hand soap,soap,lunch meat,tortillas,eggs,tortillas,paper towels,coffee/tea,toilet paper,soda,shampoo
3,lunch meat,waffles,ice cream,soda,mixes,vegetables,vegetables,waffles,dishwashing liquid/detergent,pork,bagels,lunch meat
4,all- purpose,vegetables,toilet paper,toilet paper,hand soap,bagels,milk,soda,NaN,flour,dishwashing liquid/detergent,pasta
5,flour,cheeses,dinner rolls,all- purpose,toilet paper,dishwashing liquid/detergent,ice cream,cheeses,NaN,milk,eggs,juice
6,soda,mixes,hand soap,milk,vegetables,cereals,juice,poultry,NaN,soda,sugar,bagels
7,butter,milk,spaghetti sauce,NaN,paper towels,paper towels,dishwashing liquid/detergent,toilet paper,NaN,dishwashing liquid/detergent,waffles,vegetables
8,vegetables,sandwich bags,milk,NaN,flour,butter,soap,vegetables,NaN,eggs,individual meals,laundry detergent
9,beef,laundry detergent,ketchup,NaN,pork,shampoo,sandwich bags,NaN,NaN,tortillas,toilet paper,yogurt


,transaction,product
0,1,yogurt
1,1,pork
2,1,sandwich bags
3,1,lunch meat
4,1,all- purpose
...,...,...
190,12,waffles
191,12,hand soap
192,12,butter
193,12,spaghetti sauce


In [173]:
def combs_nd(a, r, axis=0):
    a = np.asarray(a)
    if axis < 0:
        axis += a.ndim
    indices = np.arange(a.shape[axis])
    dt = np.dtype([('', np.intp)]*r)
    indices = np.fromiter(combinations(indices, r), dt)
    indices = indices.view(np.intp).reshape(-1, r)
    return np.take(a, indices, axis=axis)

In [174]:
transaction_amount = len(data_by_transaction)
threshold = 0.40

In [175]:
count_products = data_by_product['product'].value_counts()
passed_threshold = count_products[count_products.values / transaction_amount > threshold]
display(passed_threshold)
rules[1] = passed_threshold.index
pair_combs = combs_nd(passed_threshold.index, 2) 
display(pair_combs)

vegetables      11
toilet paper     9
milk             9
poultry          8
soda             7
Name: product, dtype: int64

array([['vegetables', 'toilet paper'],
       ['vegetables', 'milk'],
       ['vegetables', 'poultry'],
       ['vegetables', 'soda'],
       ['toilet paper', 'milk'],
       ['toilet paper', 'poultry'],
       ['toilet paper', 'soda'],
       ['milk', 'poultry'],
       ['milk', 'soda'],
       ['poultry', 'soda']], dtype=object)

In [176]:
def count_and_pass(pair_combs, threshold):
    pairs_freq = pd.Series(index=pair_combs)
    for pair in pair_combs:
        series_obj = data_by_transaction.apply(lambda x: True if np.isin(x.to_numpy(), pair).sum() == len(pair) else False, axis=1)
        num_of_rows = len(series_obj[series_obj == True])
        pairs_freq[tuple(pair)] = num_of_rows
    pairs_freq = pairs_freq.sort_values(ascending=False)
    display(pairs_freq)
    passed_pairs = pairs_freq[pairs_freq / len(pairs_freq) > threshold]
    return passed_pairs

In [188]:
passed_combs = count_and_pass(pair_combs, threshold)
print("Passed threshhold combs:")
display(passed_combs)
rules[2] = passed_combs.index


(vegetables, poultry)         5.0
(vegetables, milk)            5.0
(milk, poultry)               3.0
(toilet paper, soda)          3.0
(toilet paper, milk)          3.0
(vegetables, toilet paper)    3.0
(milk, soda)                  2.0
(toilet paper, poultry)       2.0
(vegetables, soda)            2.0
(poultry, soda)               1.0
dtype: float64

Passed threshhold combs:


(vegetables, poultry)    5.0
(vegetables, milk)       5.0
dtype: float64

In [187]:
flat_passed_combs = np.unique([item for sublist in passed_combs.index for item in sublist])
triple_combs = combs_nd(flat_passed_combs, 3) 
display(triple_combs)
passed_combs = count_and_pass(triple_combs, threshold)
print("Passed threshhold combs:")
display(passed_combs)
rules[3] = passed_combs.index

array([['milk', 'poultry', 'vegetables']], dtype='<U10')

(milk, poultry, vegetables)    4.0
dtype: float64

Passed threshhold combs:


(milk, poultry, vegetables)    4.0
dtype: float64

In [186]:
print("Final rules: ")
for idx, rule in enumerate(rules):
    print(f"{idx + 1}: ", rule)

Final rules: 
1:  Index(['vegetables', 'toilet paper', 'milk', 'poultry', 'soda'], dtype='object')
2:  Index([('vegetables', 'poultry'), ('vegetables', 'milk')], dtype='object')
3:  Index([('milk', 'poultry', 'vegetables')], dtype='object')
